# Foundations & Big O

*An introduction to programming concepts and algorithm analysis*


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⏱️ OTURUM ZAMANLAYICI — Bu hücreyi ilk çalıştırın!
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import time as _time, datetime as _dt, json as _json, os as _os
from IPython.display import display, HTML as _HTML

# ── Persistent tracker file (survives kernel restarts in same runtime) ──
_TRACKER_FILE = _os.path.join(
    '/content' if _os.path.isdir('/content') else '/tmp',
    '.cp2_session_Week_01.json'
)

def _load_tracker():
    """Load previous sessions from file."""
    try:
        with open(_TRACKER_FILE, 'r') as f:
            return _json.load(f)
    except (FileNotFoundError, _json.JSONDecodeError, ValueError):
        return {'sessions': [], 'total_heartbeats': 0}

def _save_tracker(data):
    """Persist tracker data to file."""
    try:
        with open(_TRACKER_FILE, 'w') as f:
            _json.dump(data, f)
    except OSError:
        pass

# ── Load any previous session data ──
_tracker = _load_tracker()
_prev_sessions = len(_tracker['sessions'])
_prev_hb = _tracker['total_heartbeats']

# ── Start new session ──
_SESSION_START = _time.time()
_SESSION_START_STR = _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
_HEARTBEATS = [_time.time()]
_CELLS_RUN = [0]
IDLE_THRESHOLD = 300   # 5 min

# Record this session start
_tracker['sessions'].append({
    'start': _SESSION_START_STR,
    'start_ts': _SESSION_START,
    'heartbeats': [_SESSION_START]
})
_save_tracker(_tracker)

def _heartbeat_hook(*args, **kwargs):
    """Record each cell execution — in-memory + file."""
    now = _time.time()
    _HEARTBEATS.append(now)
    _CELLS_RUN[0] += 1
    # Persist every 5 cells to avoid excessive disk IO
    if _CELLS_RUN[0] % 5 == 0:
        try:
            _t = _load_tracker()
            if _t['sessions']:
                _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
                _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
                _t['total_heartbeats'] = _prev_hb + len(_HEARTBEATS)
                _save_tracker(_t)
        except Exception:
            pass

def _calc_active_time(heartbeats=None):
    """Calculate active time from heartbeat list, capping idle gaps."""
    hb = heartbeats or _HEARTBEATS
    if len(hb) < 2: return 0
    active = 0
    for i in range(1, len(hb)):
        gap = hb[i] - hb[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _calc_total_active_time():
    """Calculate total across ALL sessions (multi-day support)."""
    total = 0
    try:
        _t = _load_tracker()
        for sess in _t['sessions'][:-1]:  # all previous sessions
            hb = sess.get('heartbeats', [])
            if len(hb) >= 2:
                for i in range(1, len(hb)):
                    gap = hb[i] - hb[i-1]
                    total += gap if gap <= IDLE_THRESHOLD else 30
    except Exception:
        pass
    # Add current session
    total += _calc_active_time()
    return int(total)

try:
    _ip = get_ipython()
    _ip.events.register('pre_run_cell', _heartbeat_hook)
except Exception: pass

# ── Display ──
_multi = ''
if _prev_sessions > 0:
    _multi = f'<br><span style="font-size:12px;opacity:.75">📂 {_prev_sessions} önceki oturum bulundu — çoklu oturum takibi aktif.</span>'
display(_HTML(f"""<div style='background:linear-gradient(135deg,#667eea,#764ba2);padding:14px 20px;border-radius:10px;color:white;font-family:system-ui;margin:4px 0'>
<b>⏱️ Oturum Başladı</b> — {_SESSION_START_STR}<br>
<span style='font-size:13px;opacity:.85'>Hücre aktiviteniz takip ediliyor. Bitince en alttaki Submit hücresini çalıştırın.</span>{_multi}</div>"""))
print(f'✅ Zamanlayıcı aktif. Idle eşiği: {IDLE_THRESHOLD//60} dk')
if _prev_sessions > 0:
    print(f'📂 Önceki oturumlar: {_prev_sessions} | Toplam heartbeat: {_prev_hb}')

## 🎯 Learning Objectives

- Understand what programming is and how computers execute instructions
- Write basic Python programs using variables, data types, operators, and control flow
- Create and manipulate lists and strings in Python
- Define and call functions with parameters and return values
- Analyze algorithm efficiency using Big O notation for time and space complexity

---
## 1. What is Programming?


Programming is the process of creating a set of instructions that tell a computer how to perform a task. These instructions must be precise and unambiguous because computers execute them exactly as written—they cannot infer intent or fill in missing information.

> 📖 **Definition:** A **program** (also called software or code) is a sequence of instructions written in a programming language that a computer can execute to perform a specific task or solve a problem.

Unlike humans, computers are literal interpreters. If you provide incomplete or ambiguous instructions, the computer will either fail with an error or produce incorrect results. This fundamental characteristic shapes everything about how we write programs.

> 💡 **Analogy: A Recipe**
>
> Consider baking a cake. A recipe provides step-by-step instructions:
1. Preheat oven to 180°C
2. Mix 200g flour with 150g sugar
3. Add 2 eggs and 100ml milk
4. Stir until smooth
5. Pour into pan and bake for 30 minutes
A program works similarly: it specifies exact operations in a specific order with specific data. Skip a step or change the order, and the result changes. A computer follows these instructions with perfect precision but zero understanding.

### The Stored-Program Concept

Modern computers operate on the **stored-program concept**, developed by John von Neumann in the 1940s. This revolutionary idea means that both the program (instructions) and the data it operates on are stored in the same memory. Before this, "programming" a computer meant physically rewiring it.

This concept enables the flexibility we take for granted today: the same physical computer can run a web browser, process spreadsheets, play games, or analyze scientific data—simply by loading different programs into memory.

### Levels of Abstraction

Computers fundamentally operate on **binary**—sequences of 0s and 1s representing electrical states (off/on). Writing programs directly in binary would be impractical, so we use layers of abstraction:

- **Machine Code:** Binary instructions the CPU directly executes
- **Assembly Language:** Human-readable mnemonics that map to machine code
- **High-Level Languages:** Python, Java, C++—closer to human language

Python is a **high-level language**, meaning it abstracts away hardware details. When you write `print("Hello")`, Python translates this into the many low-level operations required to display text on screen.

### Why Learn Programming?

**Problem Decomposition:** Programming teaches you to break complex problems into smaller, solvable sub-problems. This analytical skill—known as *computational thinking*—is valuable in any field.

**Automation:** Computers excel at repetitive tasks. A program can process millions of records, perform billions of calculations, or monitor systems continuously without fatigue or error.

**Precision:** Programming requires clear definitions, careful handling of edge cases, and anticipation of errors. This discipline improves logical thinking.

**Understanding Technology:** In an increasingly digital world, understanding how software works helps you make informed decisions about the technology you use daily.

### Compilation vs. Interpretation

High-level code must be translated to machine code. There are two primary approaches:

> 📖 **Definition:** **Compilation** translates the entire program to machine code before execution, producing an executable file. **Interpretation** translates and executes code line-by-line at runtime.

Python is primarily an **interpreted language**. When you run a Python program, the Python interpreter reads your code, translates it to an intermediate form called bytecode, and executes it. This makes development faster (no compilation step) but execution somewhat slower than compiled languages like C++.

---
## 2. Getting Started with Python


Python was created by Guido van Rossum and first released in 1991. The name comes from the British comedy group Monty Python, not the snake. Python emphasizes code readability and has become one of the world's most popular programming languages, used extensively in web development, data science, artificial intelligence, automation, and education.

### The Python Interpreter

Python code is executed by the **Python interpreter**. You can use Python in two modes:

- **Interactive Mode:** Type commands one at a time and see results immediately (like a calculator)
- **Script Mode:** Write a complete program in a file and run it all at once

The code blocks on this page simulate script mode—you write multiple lines and execute them together.

### The print() Function

The most basic operation is displaying output. In Python, we use the built-in `print()` function to display text or values.

> 📖 **Definition:** A **function** is a reusable block of code that performs a specific task. Functions are invoked (called) by name followed by parentheses. Values passed inside the parentheses are called **arguments**.

**Listing 1.1 — The print() Function**

In [ ]:
print("Hello, World!")
print("Welcome to Python programming.")
print("This is line three.")

***Figure 1.1:** The `print()` function outputs text. Each call produces a new line by default.*

The text `"Hello, World!"` is a **string literal**—a sequence of characters enclosed in quotation marks. Python accepts both single quotes (`'text'`) and double quotes (`"text"`); they are functionally equivalent.

### Printing Multiple Values

The `print()` function can accept multiple arguments, separated by commas. By default, it inserts a space between them:

**Listing 1.2 — Multiple Arguments**

In [ ]:
# Multiple arguments are separated by spaces
print("The answer is", 42)
print("Python", "is", "fun")

# Change the separator with sep parameter
print("2024", "01", "15", sep="-")

# Change the ending with end parameter
print("Hello", end=" ")
print("World")  # Continues on same line

***Figure 1.2:** The `sep` parameter changes the separator between arguments; `end` changes what appears at the end (default is newline).*

### Comments

Comments are notes in code that Python ignores during execution. They serve to explain what code does, why certain decisions were made, or to temporarily disable code. Good comments are essential for maintainability.

**Listing 1.3 — Comments**

In [ ]:
# This entire line is a comment - Python ignores it
print("This line executes")  # Inline comment after code

# Use comments to explain WHY, not just WHAT
# Bad:  x = x + 1  # Add 1 to x (obvious from code)
# Good: x = x + 1  # Compensate for zero-based indexing

"""
Multi-line strings (triple quotes) can also serve
as comments, though they're technically string literals
that aren't assigned to anything.
"""
print("Program continues")

***Figure 1.3:** Single-line comments use `#`. Multi-line comments can use triple quotes.*

---
## 3. Variables and Data Types


Programs need to store data for processing. **Variables** provide named references to values stored in memory, allowing us to use and manipulate data throughout our programs.

> 📖 **Definition:** A **variable** is a named storage location in computer memory that holds a value. **Assignment** is the process of binding a name to a value using the assignment operator `=`.

**Listing 1.4 — Creating Variables**

In [ ]:
# Creating variables with assignment
name = "Alice"
age = 25
height = 1.68
is_student = True

# Using variables
print("Name:", name)
print("Age:", age)
print("Height:", height, "meters")
print("Is student:", is_student)

***Figure 1.4:** Variables store values of different types. The assignment operator `=` stores the value on the right into the variable on the left.*

### Variable Naming Rules

Python has specific rules for valid variable names:

- Must start with a letter (a-z, A-Z) or underscore (_)
- Can contain letters, digits (0-9), and underscores
- Cannot be a Python reserved keyword
- Case-sensitive: `Name`, `name`, and `NAME` are three different variables

**Listing 1.5 — Valid and Invalid Names**

In [ ]:
# Valid variable names
student_name = "Bob"
_private = 100
counter2 = 0
PI = 3.14159  # Convention: ALL_CAPS for constants

# These would be INVALID (commented out):
# 2nd_place = "Silver"  # Can't start with digit
# my-variable = 5       # Hyphens not allowed
# class = "Math"        # 'class' is a reserved keyword

print(student_name, _private, counter2, PI)

***Figure 1.5:** Variable names should be descriptive. Python convention uses `snake_case` (lowercase with underscores).*

### Data Types

Every value in Python has a **type** that determines what operations can be performed on it and how it is stored in memory. Python is **dynamically typed**—you don't declare types explicitly; the interpreter infers them from values.

| Type | Description | Examples |
| --- | --- | --- |
| `int` | Integer (whole numbers, unlimited precision) | `42`, `-7`, `0`, `1000000` |
| `float` | Floating-point (decimal numbers, ~15-17 significant digits) | `3.14`, `-0.5`, `2.0`, `1e10` |
| `str` | String (text, sequence of characters) | `"hello"`, `'world'`, `""` |
| `bool` | Boolean (logical true/false) | `True`, `False` |
| `NoneType` | Represents absence of value | `None` |

**Listing 1.6 — Data Types and type()**

In [ ]:
# The type() function returns a value's type
print("Type of 42:", type(42))
print("Type of 3.14:", type(3.14))
print("Type of 'Hello':", type("Hello"))
print("Type of True:", type(True))
print("Type of None:", type(None))

# Variables take the type of their assigned value
x = 100
print("\nx =", x, "| type:", type(x))

x = "now I'm a string"
print("x =", x, "| type:", type(x))

***Figure 1.6:** The `type()` function reveals a value's type. In Python, variables can be reassigned to values of different types.*

### Type Conversion

Sometimes you need to convert values between types. Python provides built-in functions for this:

**Listing 1.7 — Type Conversion**

In [ ]:
# Converting between types
num_str = "42"
num_int = int(num_str)      # String to integer
print(f"'{num_str}' -> {num_int} (type: {type(num_int).__name__})")

pi = 3.14159
pi_int = int(pi)            # Float to int (truncates, doesn't round)
print(f"{pi} -> {pi_int} (truncated)")

value = 255
value_str = str(value)      # Integer to string
print(f"{value} -> '{value_str}' (type: {type(value_str).__name__})")

# float() converts to floating-point
print(f"float('3.14') = {float('3.14')}")
print(f"float(10) = {float(10)}")

***Figure 1.7:** Type conversion functions: `int()`, `float()`, `str()`, `bool()`.*

---
## 4. Operators and Expressions


Operators are symbols that perform operations on values (operands). An **expression** is a combination of values, variables, and operators that evaluates to a single value.

### Arithmetic Operators

| Operator | Operation | Example | Result |
| --- | --- | --- | --- |
| `+` | Addition | `7 + 3` | 10 |
| `-` | Subtraction | `7 - 3` | 4 |
| `*` | Multiplication | `7 * 3` | 21 |
| `/` | Division (always returns float) | `7 / 3` | 2.333... |
| `//` | Floor division (rounds down) | `7 // 3` | 2 |
| `%` | Modulo (remainder) | `7 % 3` | 1 |
| `**` | Exponentiation | `2 ** 10` | 1024 |

**Listing 1.8 — Arithmetic Operators**

In [ ]:
# Basic arithmetic
print("7 + 3 =", 7 + 3)
print("7 - 3 =", 7 - 3)
print("7 * 3 =", 7 * 3)
print("7 / 3 =", 7 / 3)     # True division (float result)
print("7 // 3 =", 7 // 3)   # Floor division (integer result)
print("7 % 3 =", 7 % 3)     # Modulo (remainder)
print("2 ** 10 =", 2 ** 10) # Exponentiation

# Operator precedence (PEMDAS)
result = 2 + 3 * 4          # Multiplication first
print("\n2 + 3 * 4 =", result)

result = (2 + 3) * 4        # Parentheses override
print("(2 + 3) * 4 =", result)

***Figure 1.8:** Python follows standard mathematical precedence: parentheses, exponents, multiplication/division, addition/subtraction.*

### Comparison Operators

Comparison operators compare two values and return a boolean (`True` or `False`):

| Operator | Meaning | Example | Result |
| --- | --- | --- | --- |
| `==` | Equal to | `5 == 5` | True |
| `!=` | Not equal to | `5 != 3` | True |
| `<` | Less than | `3 < 5` | True |
| `>` | Greater than | `3 > 5` | False |
| `<=` | Less than or equal | `5 <= 5` | True |
| `>=` | Greater than or equal | `5 >= 3` | True |

**Listing 1.9 — Comparison Operators**

In [ ]:
x = 10
y = 5

print(f"x = {x}, y = {y}")
print(f"x == y: {x == y}")   # Equal
print(f"x != y: {x != y}")   # Not equal
print(f"x > y: {x > y}")     # Greater than
print(f"x < y: {x < y}")     # Less than
print(f"x >= y: {x >= y}")   # Greater or equal
print(f"x <= y: {x <= y}")   # Less or equal

# Chained comparisons (Python feature)
z = 7
print(f"\n5 < {z} < 10: {5 < z < 10}")  # True

***Figure 1.9:** Comparison operators return boolean values. Note: `==` tests equality; `=` is assignment.*

### Logical Operators

Logical operators combine boolean expressions:

| Operator | Meaning | Example | Result |
| --- | --- | --- | --- |
| `and` | True if both operands are True | `True and False` | False |
| `or` | True if at least one operand is True | `True or False` | True |
| `not` | Inverts the boolean value | `not True` | False |

**Listing 1.10 — Logical Operators**

In [ ]:
age = 25
has_license = True

# Combining conditions
can_drive = age >= 18 and has_license
print(f"Age: {age}, Has license: {has_license}")
print(f"Can drive: {can_drive}")

# Or operator
is_weekend = False
is_holiday = True
day_off = is_weekend or is_holiday
print(f"\nWeekend: {is_weekend}, Holiday: {is_holiday}")
print(f"Day off: {day_off}")

# Not operator
is_raining = False
go_outside = not is_raining
print(f"\nRaining: {is_raining}, Go outside: {go_outside}")

***Figure 1.10:** Logical operators are essential for complex conditions in if statements and loops.*

### Assignment Operators

Python provides shorthand operators that combine arithmetic with assignment:

**Listing 1.11 — Assignment Operators**

In [ ]:
x = 10
print(f"Initial x: {x}")

x += 5      # Same as: x = x + 5
print(f"After x += 5: {x}")

x -= 3      # Same as: x = x - 3
print(f"After x -= 3: {x}")

x *= 2      # Same as: x = x * 2
print(f"After x *= 2: {x}")

x //= 4     # Same as: x = x // 4
print(f"After x //= 4: {x}")

x **= 2     # Same as: x = x ** 2
print(f"After x **= 2: {x}")

***Figure 1.11:** Compound assignment operators provide a concise syntax for common operations.*

---
## 5. Working with Strings


Strings are one of the most commonly used data types. Python provides powerful tools for creating and manipulating text.

> 📖 **Definition:** A **string** is an immutable sequence of characters. Once created, a string cannot be modified—operations that appear to modify strings actually create new string objects.

### Creating Strings

**Listing 1.12 — String Creation**

In [ ]:
# Single or double quotes
s1 = 'Hello'
s2 = "World"

# Quotes inside strings
s3 = "It's a beautiful day"
s4 = 'She said "Hello"'

# Triple quotes for multi-line strings
s5 = """This is a
multi-line
string."""

# Escape sequences
s6 = "Line 1\nLine 2"       # \n = newline
s7 = "Column1\tColumn2"     # \t = tab

print(s1, s2)
print(s3)
print(s4)
print(s5)
print(s6)
print(s7)

***Figure 1.12:** Strings can be created with single, double, or triple quotes. Escape sequences represent special characters.*

### String Operations

**Listing 1.13 — String Operations**

In [ ]:
# Concatenation (joining strings)
first = "Hello"
second = "World"
combined = first + " " + second
print("Concatenation:", combined)

# Repetition
line = "-" * 20
print("Repetition:", line)

# Length
message = "Python"
print(f"Length of '{message}': {len(message)}")

# Indexing (zero-based)
print(f"First character: {message[0]}")
print(f"Last character: {message[-1]}")

# Slicing [start:end:step]
print(f"First 3: {message[0:3]}")    # 'Pyt'
print(f"Last 3: {message[-3:]}")     # 'hon'
print(f"Every 2nd: {message[::2]}")  # 'Pto'
print(f"Reversed: {message[::-1]}")  # 'nohtyP'

***Figure 1.13:** String operations include concatenation, repetition, indexing, and slicing.*

### String Methods

Strings have many built-in methods (functions that belong to an object):

**Listing 1.14 — String Methods**

In [ ]:
text = "  Hello, World!  "

# Case methods
print(text.upper())          # "  HELLO, WORLD!  "
print(text.lower())          # "  hello, world!  "
print("hello".capitalize())  # "Hello"
print("hello world".title()) # "Hello World"

# Whitespace
print(f"'{text.strip()}'")   # Remove leading/trailing whitespace

# Find and replace
print(text.find("World"))    # Returns index: 9
print(text.replace("World", "Python"))

# Check content
print("hello".isalpha())     # True (all letters)
print("12345".isdigit())     # True (all digits)
print("hello".startswith("he"))  # True
print("hello".endswith("lo"))    # True

# Split and join
words = "apple,banana,cherry".split(",")
print(words)                 # ['apple', 'banana', 'cherry']
print("-".join(words))       # "apple-banana-cherry"

***Figure 1.14:** String methods provide powerful text processing capabilities.*

### Formatted Strings (f-strings)

Python 3.6 introduced f-strings, the most readable way to embed expressions in strings:

**Listing 1.15 — F-strings**

In [ ]:
name = "Alice"
age = 30
height = 1.756

# Basic f-string
print(f"Name: {name}, Age: {age}")

# Expressions inside f-strings
print(f"Next year: {age + 1}")
print(f"Name uppercase: {name.upper()}")

# Formatting numbers
print(f"Height: {height:.2f} meters")  # 2 decimal places
print(f"Large number: {1000000:,}")    # Thousands separator

# Padding and alignment
print(f"{'left':<10}|{'center':^10}|{'right':>10}")

# Percentage
ratio = 0.756
print(f"Success rate: {ratio:.1%}")    # 75.6%

***Figure 1.15:** F-strings allow embedded expressions and sophisticated formatting. They're prefixed with `f` before the quotes.*

---
## 6. Control Flow


By default, Python executes statements sequentially from top to bottom. **Control flow** statements allow us to change this order—executing code conditionally or repeatedly.

### Conditional Statements: if, elif, else

> 📖 **Definition:** A **conditional statement** executes a block of code only if a specified condition evaluates to `True`. Python uses `if`, `elif` (else if), and `else` keywords.

**Listing 1.16 — If Statements**

In [ ]:
score = 85

# Simple if
if score >= 60:
    print("Passed!")

# if-else
if score >= 90:
    print("Excellent!")
else:
    print("Keep working!")

# if-elif-else chain
if score >= 90:
    grade = "A"
elif score >= 80:
    grade = "B"
elif score >= 70:
    grade = "C"
elif score >= 60:
    grade = "D"
else:
    grade = "F"

print(f"Score: {score}, Grade: {grade}")

***Figure 1.16:** Conditional statements use indentation to define code blocks. Python requires consistent indentation (typically 4 spaces).*

### Loops: while

A `while` loop repeats a block of code as long as a condition remains true:

**Listing 1.17 — While Loops**

In [ ]:
# Count from 1 to 5
count = 1
while count <= 5:
    print(f"Count: {count}")
    count += 1

print("Done!")

# Sum numbers until total exceeds 20
total = 0
num = 1
while total <= 20:
    total += num
    num += 1

print(f"\nSum exceeded 20 at total = {total}")

***Figure 1.17:** While loops continue as long as the condition is True. Be careful to avoid infinite loops!*

### Loops: for

A `for` loop iterates over a sequence (list, string, range, etc.):

**Listing 1.18 — For Loops**

In [ ]:
# Iterate over a list
fruits = ["apple", "banana", "cherry"]
for fruit in fruits:
    print(f"I like {fruit}")

# Iterate over a string
print("\nLetters in 'Python':")
for char in "Python":
    print(char, end=" ")

# Using range()
print("\n\nNumbers 0-4:")
for i in range(5):           # 0, 1, 2, 3, 4
    print(i, end=" ")

print("\n\nNumbers 2-6:")
for i in range(2, 7):        # 2, 3, 4, 5, 6
    print(i, end=" ")

print("\n\nEven numbers 0-10:")
for i in range(0, 11, 2):    # 0, 2, 4, 6, 8, 10
    print(i, end=" ")

***Figure 1.18:** The `range(start, stop, step)` function generates sequences of integers. Note: `stop` is exclusive.*

### Loop Control: break and continue

**Listing 1.19 — Break and Continue**

In [ ]:
# break: exit loop entirely
print("Finding first even number:")
for num in [1, 3, 5, 4, 7, 2]:
    if num % 2 == 0:
        print(f"Found: {num}")
        break
    print(f"Checking {num}...")

# continue: skip to next iteration
print("\nPrinting odd numbers only:")
for num in range(10):
    if num % 2 == 0:
        continue    # Skip even numbers
    print(num, end=" ")

***Figure 1.19:** `break` exits the loop completely; `continue` skips to the next iteration.*

---
## 7. Collections: Lists


Programs often work with groups of related values. The **list** is Python's most versatile built-in collection type.

> 📖 **Definition:** A **list** is an ordered, mutable (changeable) sequence of values. Lists are created with square brackets `[]`, elements separated by commas. Lists can contain any data types, including other lists.

### Creating and Accessing Lists

**Listing 1.20 — List Basics**

In [ ]:
# Creating lists
numbers = [10, 20, 30, 40, 50]
mixed = [1, "two", 3.0, True, None]
empty = []
nested = [[1, 2], [3, 4], [5, 6]]

# Accessing elements (zero-based indexing)
print(f"First: {numbers[0]}")      # 10
print(f"Third: {numbers[2]}")      # 30
print(f"Last: {numbers[-1]}")      # 50
print(f"Second to last: {numbers[-2]}")  # 40

# Nested access
print(f"Nested [1][0]: {nested[1][0]}")  # 3

# Length
print(f"Length: {len(numbers)}")

***Figure 1.20:** Lists use zero-based indexing. The first element is at index 0; negative indices count from the end.*

### List Slicing

**Listing 1.21 — List Slicing**

In [ ]:
nums = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Slicing: [start:stop:step]
print(f"Original: {nums}")
print(f"[2:7]: {nums[2:7]}")       # Elements 2-6
print(f"[:5]: {nums[:5]}")         # First 5
print(f"[5:]: {nums[5:]}")         # From index 5 to end
print(f"[::2]: {nums[::2]}")       # Every 2nd element
print(f"[1::2]: {nums[1::2]}")     # Every 2nd, starting at 1
print(f"[::-1]: {nums[::-1]}")     # Reversed

# Slicing creates a copy
original = [1, 2, 3]
copy = original[:]
copy[0] = 999
print(f"\nOriginal: {original}, Copy: {copy}")

***Figure 1.21:** Slicing syntax: `[start:stop:step]`. The `stop` index is exclusive. Omitted values default to start=0, stop=len, step=1.*

### Modifying Lists

**Listing 1.22 — List Modification**

In [ ]:
fruits = ["apple", "banana", "cherry"]
print(f"Original: {fruits}")

# Modify element
fruits[1] = "blueberry"
print(f"After fruits[1] = 'blueberry': {fruits}")

# append(): add to end - O(1) amortized
fruits.append("date")
print(f"After append('date'): {fruits}")

# insert(): add at position - O(n)
fruits.insert(1, "apricot")
print(f"After insert(1, 'apricot'): {fruits}")

# remove(): remove by value - O(n)
fruits.remove("cherry")
print(f"After remove('cherry'): {fruits}")

# pop(): remove and return by index - O(1) for last, O(n) for others
last = fruits.pop()
print(f"After pop(): {fruits}, removed: {last}")

first = fruits.pop(0)
print(f"After pop(0): {fruits}, removed: {first}")

***Figure 1.22:** Lists are mutable—elements can be added, removed, or changed. Time complexities are noted for each operation.*

### List Operations and Methods

**Listing 1.23 — List Operations**

In [ ]:
numbers = [3, 1, 4, 1, 5, 9, 2, 6]

# Membership test - O(n)
print(f"5 in list: {5 in numbers}")
print(f"7 in list: {7 in numbers}")

# Count occurrences
print(f"Count of 1: {numbers.count(1)}")

# Find index (raises ValueError if not found)
print(f"Index of 5: {numbers.index(5)}")

# Sort (modifies in place)
numbers.sort()
print(f"Sorted: {numbers}")

numbers.sort(reverse=True)
print(f"Sorted descending: {numbers}")

# Reverse (modifies in place)
numbers.reverse()
print(f"Reversed: {numbers}")

# Aggregation functions
nums = [10, 20, 30, 40, 50]
print(f"\nsum: {sum(nums)}, min: {min(nums)}, max: {max(nums)}")

***Figure 1.23:** Common list operations and built-in functions for aggregation.*

### List Comprehensions

List comprehensions provide a concise way to create lists:

**Listing 1.24 — List Comprehensions**

In [ ]:
# Traditional loop approach
squares = []
for x in range(1, 6):
    squares.append(x ** 2)
print(f"Traditional: {squares}")

# List comprehension (equivalent, more concise)
squares = [x ** 2 for x in range(1, 6)]
print(f"Comprehension: {squares}")

# With condition
evens = [x for x in range(10) if x % 2 == 0]
print(f"Even numbers: {evens}")

# Transform and filter
words = ["hello", "WORLD", "Python", "AI"]
lower_long = [w.lower() for w in words if len(w) > 3]
print(f"Lowercase, length > 3: {lower_long}")

***Figure 1.24:** List comprehensions: `[expression for item in iterable if condition]`. They're often faster and more readable than explicit loops.*

---
## 8. Introduction to Functions


Functions are reusable blocks of code that perform specific tasks. They help organize code, avoid repetition, and make programs easier to understand and maintain.

> 📖 **Definition:** A **function** is a named sequence of statements that performs a computation. Functions accept **parameters** (input) and may **return** a value (output).

### Defining Functions

**Listing 1.25 — Function Definition**

In [ ]:
# Function with no parameters, no return
def greet():
    print("Hello, World!")

greet()  # Call the function

# Function with parameter
def greet_person(name):
    print(f"Hello, {name}!")

greet_person("Alice")
greet_person("Bob")

# Function with return value
def square(x):
    return x ** 2

result = square(5)
print(f"5 squared is {result}")

***Figure 1.25:** Functions are defined with `def`, followed by the name, parameters in parentheses, and a colon. The body is indented.*

### Parameters and Arguments

**Listing 1.26 — Function Parameters**

In [ ]:
# Multiple parameters
def add(a, b):
    return a + b

print(f"3 + 5 = {add(3, 5)}")

# Default parameter values
def greet(name, greeting="Hello"):
    return f"{greeting}, {name}!"

print(greet("Alice"))              # Uses default
print(greet("Bob", "Hi"))          # Override default
print(greet(greeting="Hey", name="Charlie"))  # Keyword args

# Multiple return values (returns a tuple)
def min_max(numbers):
    return min(numbers), max(numbers)

minimum, maximum = min_max([3, 1, 4, 1, 5, 9])
print(f"Min: {minimum}, Max: {maximum}")

***Figure 1.26:** Parameters can have default values. Arguments can be passed by position or keyword.*

### Docstrings

**Listing 1.27 — Function Documentation**

In [ ]:
def calculate_area(length, width):
    """
    Calculate the area of a rectangle.
    
    Args:
        length: The length of the rectangle (positive number)
        width: The width of the rectangle (positive number)
    
    Returns:
        The area (length * width)
    
    Example:
        >>> calculate_area(5, 3)
        15
    """
    return length * width

# Access the docstring
print(calculate_area.__doc__)

# Use the function
print(f"\nArea: {calculate_area(10, 5)}")

***Figure 1.27:** Docstrings document what a function does. They're accessed via the `__doc__` attribute or `help()` function.*

---
## 9. Algorithm Analysis: Big O Notation


When solving problems with code, we care about both *correctness* (does it produce the right answer?) and *efficiency* (how much time and memory does it use?). Efficiency becomes critical when working with large data sets.

> 📖 **Definition:** An **algorithm** is a step-by-step procedure for solving a problem. Different algorithms for the same problem may have vastly different efficiencies.

### Why Analyze Algorithms?

We cannot measure efficiency simply by timing execution—results depend on hardware, other running processes, and specific inputs. Instead, we analyze how resource usage grows as input size increases.

> 💡 **Example: Searching a Phone Book**
>
> Suppose you need to find a name in a phone book with n entries:
- **Linear Search:** Start at page 1, check each entry. On average, check n/2 entries.
- **Binary Search:** Open to the middle, determine which half contains the name, repeat. Check at most log₂(n) entries.
For n = 1,000,000: linear search averages 500,000 checks; binary search needs at most 20 checks. The difference is dramatic!

> 📖 **Definition:** **Big O notation** describes the upper bound of an algorithm's growth rate as a function of input size n. It characterizes how performance scales, ignoring constant factors and lower-order terms.

### Why Ignore Constants?

Big O focuses on scalability. An algorithm that takes 2n steps and one that takes 100n steps are both O(n)—they both scale linearly. As n grows large, the growth rate dominates any constant factor.

Consider two algorithms for processing n items:

- Algorithm A: 100n operations (O(n))
- Algorithm B: 0.01n² operations (O(n²))

For n = 100: A does 10,000 ops, B does 100 ops. B wins.

For n = 100,000: A does 10,000,000 ops, B does 100,000,000,000 ops. A wins by 10,000x.

### Common Time Complexities

| Big O | Name | Description | Example |
| --- | --- | --- | --- |
| O(1) | Constant | Same time regardless of input | Array index access |
| O(log n) | Logarithmic | Halves problem each step | Binary search |
| O(n) | Linear | Examines each element once | Linear search |
| O(n log n) | Linearithmic | Divide-and-conquer sorts | Merge sort |
| O(n²) | Quadratic | Nested loops | Bubble sort |
| O(2ⁿ) | Exponential | Doubles each step | Naive Fibonacci |

### Growth Rate Comparison

The following table shows approximate operation counts:

| n | O(1) | O(log n) | O(n) | O(n log n) | O(n²) |
| --- | --- | --- | --- | --- | --- |
| 10 | 1 | 3 | 10 | 33 | 100 |
| 100 | 1 | 7 | 100 | 664 | 10,000 |
| 1,000 | 1 | 10 | 1,000 | 9,966 | 1,000,000 |
| 10,000 | 1 | 13 | 10,000 | 132,877 | 100,000,000 |
| 1,000,000 | 1 | 20 | 1,000,000 | 19,931,569 | 10¹² |

If each operation takes 1 microsecond:

- O(n) with n = 1,000,000: ~1 second
- O(n²) with n = 1,000,000: ~12 days
- O(2ⁿ) with n = 50: ~35 years

### Analyzing Code: O(1) — Constant Time

Operations that take the same time regardless of input size:

**Listing 1.28 — O(1) Examples**

In [ ]:
# O(1) operations
data = [10, 20, 30, 40, 50]

# Array index access - O(1)
first = data[0]
last = data[-1]
print(f"First: {first}, Last: {last}")

# Length of list - O(1) (Python caches this)
size = len(data)
print(f"Size: {size}")

# Dictionary lookup - O(1) average
phone_book = {"Alice": "555-1234", "Bob": "555-5678"}
alice_phone = phone_book["Alice"]
print(f"Alice's phone: {alice_phone}")

# Append to list - O(1) amortized
data.append(60)
print(f"After append: {data}")

***Figure 1.28:** O(1) operations execute in constant time—performance doesn't depend on data size.*

### Analyzing Code: O(n) — Linear Time

**Listing 1.29 — O(n) Examples**

In [ ]:
# O(n) - Linear time algorithms

def linear_search(data, target):
    """Search for target in data. O(n)"""
    for i, value in enumerate(data):
        if value == target:
            return i
    return -1

def find_sum(data):
    """Calculate sum of all elements. O(n)"""
    total = 0
    for value in data:
        total += value
    return total

def find_max(data):
    """Find maximum value. O(n)"""
    if not data:
        return None
    maximum = data[0]
    for value in data[1:]:
        if value > maximum:
            maximum = value
    return maximum

numbers = [15, 42, 8, 23, 4, 16, 55, 31]
print(f"List: {numbers}")
print(f"Index of 16: {linear_search(numbers, 16)}")
print(f"Sum: {find_sum(numbers)}")
print(f"Max: {find_max(numbers)}")

***Figure 1.29:** O(n) algorithms examine each element at most a constant number of times.*

### Analyzing Code: O(n²) — Quadratic Time

**Listing 1.30 — O(n²) Examples**

In [ ]:
# O(n²) - Quadratic time algorithms

def has_duplicates_naive(data):
    """Check for duplicates by comparing all pairs. O(n²)"""
    n = len(data)
    comparisons = 0
    for i in range(n):
        for j in range(i + 1, n):
            comparisons += 1
            if data[i] == data[j]:
                return True, comparisons
    return False, comparisons

def bubble_sort(data):
    """Simple but inefficient sorting. O(n²)"""
    data = data.copy()  # Don't modify original
    n = len(data)
    for i in range(n):
        for j in range(0, n - i - 1):
            if data[j] > data[j + 1]:
                data[j], data[j + 1] = data[j + 1], data[j]
    return data

test = [3, 1, 4, 1, 5]
has_dup, comps = has_duplicates_naive(test)
print(f"List: {test}")
print(f"Has duplicates: {has_dup} (found in {comps} comparisons)")

unsorted = [64, 34, 25, 12, 22, 11, 90]
print(f"\nUnsorted: {unsorted}")
print(f"Sorted: {bubble_sort(unsorted)}")

***Figure 1.30:** O(n²) algorithms often involve nested loops. They become impractical for large inputs.*

### Space Complexity

Big O also describes memory usage. An algorithm is:

- **O(1) space:** Uses constant extra memory (e.g., only a few variables)
- **O(n) space:** Uses memory proportional to input size (e.g., copies the input)

**Listing 1.31 — Space Complexity**

In [ ]:
# O(1) space - uses constant extra memory
def sum_in_place(data):
    total = 0  # Only one extra variable
    for x in data:
        total += x
    return total

# O(n) space - creates new list
def double_values(data):
    result = []  # New list grows with input
    for x in data:
        result.append(x * 2)
    return result

# Also O(n) space (list comprehension)
def double_values_v2(data):
    return [x * 2 for x in data]

nums = [1, 2, 3, 4, 5]
print(f"Sum (O(1) space): {sum_in_place(nums)}")
print(f"Doubled (O(n) space): {double_values(nums)}")

***Figure 1.31:** Space complexity measures additional memory used, not including the input itself.*

- Big O describes how algorithm performance scales with input size
- It captures worst-case growth rate, ignoring constant factors
- O(1) < O(log n) < O(n) < O(n log n) < O(n²) < O(2ⁿ)
- A single loop over n elements is typically O(n)
- Nested loops over n elements are typically O(n²)
- Choosing appropriate data structures and algorithms dramatically affects performance
- Space complexity measures memory usage, not just time

### Amortized Analysis

Some operations have different costs each time they run. For example, appending to a Python list is usually O(1), but occasionally the list must resize its underlying array, which costs O(n). **Amortized analysis** gives the average cost per operation over a worst-case sequence of operations.

> 📖 **Definition:** **Amortized analysis** determines the average performance of each operation in a worst-case sequence, ensuring that the total cost of n operations is bounded even though individual operations may occasionally be expensive.

**Listing 1.32 — Amortized Analysis: Dynamic Array**

In [ ]:
# Amortized Analysis: Why list.append() is O(1) amortized

class DynamicArray:
    """
    Simulates how Python lists grow internally.
    When full, capacity doubles — the resize is O(n),
    but it happens so rarely that append is O(1) amortized.
    """
    def __init__(self):
        self.data = [None] * 1  # Start with capacity 1
        self.size = 0
        self.capacity = 1
        self.resize_count = 0
        self.total_copies = 0

    def append(self, value):
        if self.size == self.capacity:
            # Resize: double the capacity (this is O(n))
            self.resize_count += 1
            self.total_copies += self.size
            new_data = [None] * (self.capacity * 2)
            for i in range(self.size):
                new_data[i] = self.data[i]
            self.data = new_data
            self.capacity *= 2
        self.data[self.size] = value
        self.size += 1

# Demonstrate: append 1000 elements
arr = DynamicArray()
for i in range(1000):
    arr.append(i)

print(f"After 1000 appends:")
print(f"  Resizes needed:     {arr.resize_count}")
print(f"  Total element copies: {arr.total_copies}")
print(f"  Average copies per append: {arr.total_copies / 1000:.2f}")
print(f"  Final capacity:     {arr.capacity}")
print()
print("Key insight: 1000 appends needed only ~10 resizes.")
print("Total work = n + n/2 + n/4 + ... < 2n = O(n)")
print("So each append costs O(n)/n = O(1) amortized!")

***Figure 1.32:** Dynamic array doubling strategy makes append O(1) amortized. Although occasional resizes are O(n), they happen exponentially less frequently.*

---
# 📝 Exercises


Complete the following exercises to reinforce your understanding. Each exercise shows the expected answer—use the code cell below it to write your solution.

### Exercise 1: Temperature Conversion  (⭐ Easy)

Write a function `celsius_to_fahrenheit(celsius)` that converts a temperature from Celsius to Fahrenheit using the formula: F = C × 9/5 + 32. Test it with the values 0, 100, and 37.

**Expected:** (Expected: 0°C = 32.0°F, 100°C = 212.0°F, 37°C = 98.6°F)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Function Definition:**

                        Define a function using `def function_name(parameter):`

                        Example: `def celsius_to_fahrenheit(celsius):`
- **Hint 2 - The Formula:**

                        Inside the function, calculate: `fahrenheit = celsius * 9/5 + 32`

                        Then use `return fahrenheit` to return the result.
- **Hint 3 - Testing:**

                        Call your function with test values and print the results:

`print(f"0°C = {celsius_to_fahrenheit(0)}°F")`
</details>

In [ ]:
# ✏️ [EX1]
# Temperature Conversion
# Implement the celsius_to_fahrenheit function

def celsius_to_fahrenheit(celsius):
    # Your code here: implement the formula F = C × 9/5 + 32
    pass  # Remove this line when you add your code

# Test your function (uncomment these lines)
# print(f"0°C = {celsius_to_fahrenheit(0)}°F")
# print(f"100°C = {celsius_to_fahrenheit(100)}°F")
# print(f"37°C = {celsius_to_fahrenheit(37)}°F")

### Exercise 2: List Statistics  (⭐ Easy)

Given the list `scores = [85, 92, 78, 90, 88, 76, 95, 89]`, write code to find and print: the number of scores, the sum, the average (mean), the minimum, and the maximum.

**Expected:** (Expected: Count=8, Sum=693, Average=86.625, Min=76, Max=95)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Built-in Functions:**

                        Python has built-in functions: `len()`, `sum()`, `min()`, `max()`

                        Example: `count = len(scores)`
- **Hint 2 - Calculating Average:**

                        Average = sum divided by count

`average = sum(scores) / len(scores)`
- **Hint 3 - Min and Max:**

`minimum = min(scores)`

`maximum = max(scores)`
</details>

In [ ]:
# ✏️ [EX2]
scores = [85, 92, 78, 90, 88, 76, 95, 89]

# Calculate the statistics using Python built-in functions
count = None    # Replace None with your code
total = None    # Replace None with your code
average = None  # Replace None with your code
minimum = None  # Replace None with your code
maximum = None  # Replace None with your code

print(f"Count: {count}")
print(f"Sum: {total}")
print(f"Average: {average}")
print(f"Min: {minimum}")
print(f"Max: {maximum}")

### Exercise 3: FizzBuzz  (⭐ Easy)

Write a program that prints numbers from 1 to 20, but: for multiples of 3, print "Fizz" instead; for multiples of 5, print "Buzz" instead; for multiples of both 3 and 5, print "FizzBuzz".

**Expected:** (Expected: 1, 2, Fizz, 4, Buzz, Fizz, 7, 8, Fizz, Buzz, 11, Fizz, 13, 14, FizzBuzz, 16, 17, Fizz, 19, Buzz)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Loop Structure:**

                        Use `for num in range(1, 21):` to loop from 1 to 20.

                        The modulo operator `%` checks divisibility: `num % 3 == 0`
- **Hint 2 - Order Matters!**

                        Check "divisible by both 3 AND 5" FIRST!

`if num % 3 == 0 and num % 5 == 0:`

                        Then check 3 only, then 5 only, then else.
- **Hint 3 - Print Formatting:**

                        Use `print(..., end=" ")` to keep output on the same line.

                        Or use regular print for each on new line.
</details>

In [ ]:
# ✏️ [EX3]
# FizzBuzz Challenge
# Print numbers 1-20, but Fizz for multiples of 3, Buzz for 5, FizzBuzz for both

for num in range(1, 21):
    # Your code here: use if/elif/else to check divisibility
    pass  # Remove this and add your logic

### Exercise 4: Count Vowels  (⭐⭐ Medium)

Write a function `count_vowels(text)` that returns the number of vowels (a, e, i, o, u) in a string. Test it with "Hello World" and "Python Programming".

**Expected:** (Expected: "Hello World" has 3 vowels, "Python Programming" has 4 vowels)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Define Vowels:**

                        Create a string containing all vowels: `vowels = "aeiouAEIOU"`

                        Or convert text to lowercase: `text.lower()` and use `"aeiou"`
- **Hint 2 - Loop Through Characters:**

                        Initialize a counter: `count = 0`

                        Loop through each character: `for char in text:`
- **Hint 3 - Check Membership:**

                        Use the `in` operator to check if character is a vowel:

`if char in vowels:` then `count += 1`
</details>

In [ ]:
# ✏️ [EX4]
# Count Vowels Function
# Return the number of vowels (a, e, i, o, u) in a string

def count_vowels(text):
    # Your code here
    pass  # Remove this line

# Test your function (uncomment these lines)
# print(f'"Hello World" has {count_vowels("Hello World")} vowels')
# print(f'"Python Programming" has {count_vowels("Python Programming")} vowels')

### Exercise 5: Big O Analysis  (⭐⭐ Medium)

Analyze the time complexity of each function below. Fill in the Big O notation in the comments and explain why.

**Expected:** (Expected: func_a is O(1), func_b is O(n), func_c is O(n²), func_d is O(n))

<details>
<summary>💡 Hints</summary>

- **Hint 1 - O(1) Constant Time:**

                        Operations that don't depend on input size are O(1).

                        Example: Accessing an element by index `data[0]` is O(1).
- **Hint 2 - O(n) Linear Time:**

                        A single loop over n elements is O(n).

`for x in data:` iterates n times → O(n)
- **Hint 3 - O(n²) Quadratic Time:**

                        Nested loops each running n times → n × n = O(n²)

`for i in range(n): for j in range(n):` → O(n²)
</details>

In [ ]:
# ✏️ [EX5]
# Big O Analysis
# Update the docstrings with the correct time complexity

def func_a(data):
    """Time Complexity: O(???) - Why: ???"""
    return data[0] + data[-1]

def func_b(data):
    """Time Complexity: O(???) - Why: ???"""
    total = 0
    for x in data:
        total += x
    return total

def func_c(data):
    """Time Complexity: O(???) - Why: ???"""
    n = len(data)
    count = 0
    for i in range(n):
        for j in range(n):
            count += 1
    return count

def func_d(data):
    """Time Complexity: O(???) - Why: ???"""
    result = []
    for x in data:
        result.append(x * 2)
    return result

# Test with a sample list
test_data = [1, 2, 3, 4, 5]
print(f"func_a result: {func_a(test_data)}")
print(f"func_b result: {func_b(test_data)}")
print(f"func_c result: {func_c(test_data)}")
print(f"func_d result: {func_d(test_data)}")

print("\nUpdate the docstrings with your Big O answers!")

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,12}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-12 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
_ORIGINAL_HASHES = {}

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request
import time as _time, datetime as _dt

WEEK = "Week_01"
URL  = "https://script.google.com/macros/s/AKfycbxepk2NvNg3Whad-WOPxdZI-mWnVJeNKCsZVspvk7Ku5YHC_oWv7376VrWLn_30nyI_vw/exec"

# ══════════════════════════════════════════════════════════
# SELF-HEALING TIMER — works even if Cell 1 was skipped
# ══════════════════════════════════════════════════════════
_TRACKER_FILE = os.path.join(
    '/content' if os.path.isdir('/content') else '/tmp',
    f'.cp2_session_{WEEK}.json'
)
IDLE_THRESHOLD = 300

def _sh_calc_active(heartbeats):
    """Calculate active seconds from a heartbeat list."""
    if len(heartbeats) < 2: return 0
    active = 0
    for i in range(1, len(heartbeats)):
        gap = heartbeats[i] - heartbeats[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _collect_timing():
    """
    Collect timing data from all available sources:
      1. In-memory variables (timer cell ran in this kernel)
      2. Persistent tracker file (previous sessions / kernel restarts)
      3. Minimal fallback (nothing available)
    """
    _info = {
        'active_time': 0, 'wall_time': 0, 'cells_run': 0,
        'heartbeat_count': 0, 'session_count': 0,
        'session_start': '', 'session_end': _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'source': 'none'
    }

    # ── Source 1: In-memory (timer cell ran this kernel) ──
    _has_memory = False
    try:
        _s = _SESSION_START        # noqa: F821
        _hb = list(_HEARTBEATS)    # noqa: F821
        _cr = _CELLS_RUN[0]        # noqa: F821
        _has_memory = True
        _info['active_time'] = _sh_calc_active(_hb)
        _info['wall_time'] = int(_time.time() - _s)
        _info['cells_run'] = _cr
        _info['heartbeat_count'] = len(_hb)
        _info['session_start'] = _SESSION_START_STR  # noqa: F821
        _info['source'] = 'memory'
    except NameError:
        pass

    # ── Source 2: Persistent file (adds previous sessions) ──
    try:
        with open(_TRACKER_FILE, 'r') as f:
            _t = json.load(f)
        sessions = _t.get('sessions', [])
        _info['session_count'] = len(sessions)

        if not _has_memory and sessions:
            # Timer cell was NOT run — reconstruct from file
            last = sessions[-1]
            hb = last.get('heartbeats', [])
            _info['active_time'] = _sh_calc_active(hb)
            _info['wall_time'] = int(hb[-1] - hb[0]) if len(hb) >= 2 else 0
            _info['cells_run'] = last.get('cells_run', len(hb))
            _info['heartbeat_count'] = len(hb)
            _info['session_start'] = last.get('start', '')
            _info['source'] = 'file-current'

        # Add previous sessions' active time
        if len(sessions) > 1:
            prev_active = 0
            prev_cells = 0
            prev_hb = 0
            cutoff = -1 if _has_memory else -1  # exclude current session
            for sess in sessions[:cutoff]:
                hb = sess.get('heartbeats', [])
                prev_active += _sh_calc_active(hb)
                prev_cells += sess.get('cells_run', len(hb))
                prev_hb += len(hb)
            _info['prev_active_time'] = prev_active
            _info['prev_cells_run'] = prev_cells
            _info['prev_heartbeats'] = prev_hb
            _info['total_active_time'] = _info['active_time'] + prev_active
            _info['total_cells_run'] = _info['cells_run'] + prev_cells
            if _info['source'] == 'none':
                _info['source'] = 'file-prev'

    except (FileNotFoundError, json.JSONDecodeError, KeyError, ValueError):
        pass

    # Ensure totals exist
    _info.setdefault('total_active_time', _info['active_time'])
    _info.setdefault('total_cells_run', _info['cells_run'])
    return _info

# ── Persist final heartbeat snapshot before collecting ──
try:
    _t = json.load(open(_TRACKER_FILE))
    if _t['sessions']:
        _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
        _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
        json.dump(_t, open(_TRACKER_FILE, 'w'))
except Exception:
    pass

_timing = _collect_timing()

# ── Display timing ──
if _timing['source'] == 'none':
    print("⚠️  Zamanlayıcı verisi bulunamadı — ilk hücreyi çalıştırmayı unutmayın.")
    print("    Gönderim yine de yapılacak (süre: 0 olarak kaydedilir).")
else:
    _a = _timing['active_time']
    _w = _timing['wall_time']
    print(f"⏱️  Bu oturum — Aktif: {_a//60}m {_a%60}s  |  Duvar: {_w//60}m {_w%60}s")
    print(f"🔢  Hücreler: {_timing['cells_run']}  |  Heartbeat: {_timing['heartbeat_count']}")
    if _timing['session_count'] > 1:
        _ta = _timing['total_active_time']
        print(f"📂  Toplam {_timing['session_count']} oturum — Toplam aktif: {_ta//60}m {_ta%60}s  |  Toplam hücre: {_timing['total_cells_run']}")
    if _timing['total_active_time'] < 120:
        print("⚠️  Toplam aktif süre < 2 dk — çalıştırdığınız hücre sayısı düşük olabilir.")

_time_on_page = _timing['total_active_time']

# ══════════════════════════════════════════════════════════
# STUDENT INFO CHECK
# ══════════════════════════════════════════════════════════
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın (öğrenci bilgileri).")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın — bazı alanlar boş.")

# ══════════════════════════════════════════════════════════
# EXERCISE ANSWER COLLECTION (improved with retry)
# ══════════════════════════════════════════════════════════
_EX_PATTERN = re.compile(r'#\s*✏️\s*\[EX(\d+)\]')

def _extract_exercise(src):
    """Try to extract exercise ID and code from a source string."""
    m = _EX_PATTERN.match(src)
    if not m: return None, None
    ex_id = 'ex' + m.group(1)
    clean = '\n'.join(src.split('\n')[1:]).strip()
    return ex_id, clean

_answers = {}

# ── Method 1: In[] list (most reliable for Run-All) ──
try:
    for _src in In:
        if not _src: continue
        _eid, _code = _extract_exercise(_src)
        if _eid:
            _answers[_eid] = {
                'code': _code,
                'modified': len(_code) > 5
            }
except NameError:
    pass

# ── Method 2: IPython history_manager (flush first!) ──
if not _answers:
    try:
        _ipy = get_ipython()
        # Force flush so Run-All history is fully written
        try: _ipy.history_manager.db.commit()
        except Exception: pass
        _time.sleep(0.5)  # small delay for DB sync
        for _sess, _line, _src in _ipy.history_manager.get_range(output=False):
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': len(_code) > 5
                }
    except Exception:
        pass

# ── Method 3: Read notebook file (VS Code / local Jupyter) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir('.') if f.endswith('.ipynb') and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), 'r', encoding='utf-8') as _f:
            _nb = json.load(_f)
        for _cell in _nb['cells']:
            if _cell['cell_type'] != 'code': continue
            _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': len(_code) > 5
                }

# ── Method 4: Colab notebook file at /content/ ──
if not _answers:
    try:
        _colab_candidates = [f for f in os.listdir('/content') if f.endswith('.ipynb') and WEEK in f]
        if _colab_candidates:
            _nb_path = os.path.join('/content', _colab_candidates[0])
            with open(_nb_path, 'r', encoding='utf-8') as _f:
                _nb = json.load(_f)
            for _cell in _nb['cells']:
                if _cell['cell_type'] != 'code': continue
                _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
                _eid, _code = _extract_exercise(_src)
                if _eid:
                    _answers[_eid] = {
                        'code': _code,
                        'modified': len(_code) > 5
                    }
    except Exception:
        pass

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys(), key=lambda x: int(x[2:]) if x[2:].isdigit() else 999))}")

if not _answers:
    print("\n⚠️  Hiçbir exercise yanıtı bulunamadı!")
    print("Submit'ten ÖNCE tüm exercise hücrelerini çalıştırdığınızdan emin olun.")
    print("\n💡 İpucu: 'Run All' yerine exercise hücrelerini tek tek çalıştırıp")
    print("   ardından bu hücreyi çalıştırmayı deneyin.")
    raise SystemExit()

# ══════════════════════════════════════════════════════════
# SEND SUBMISSION
# ══════════════════════════════════════════════════════════
_data = json.dumps({
    'week': WEEK,
    'studentId': _sid,
    'studentName': _sname,
    'studentEmail': _semail,
    'classCode': _scode,
    'source': 'dsa-notebook',
    'timeOnPage': _time_on_page,
    'wallTime': _timing['wall_time'],
    'cellsRun': _timing['total_cells_run'],
    'sessionCount': _timing['session_count'],
    'sessionStart': _timing['session_start'],
    'sessionEnd': _timing['session_end'],
    'timerSource': _timing['source'],
    'answers': _answers
}).encode('utf-8')

print("📡 Gönderiliyor...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get('success'):
        print(f"\n✅ {_result['message']}")
        print('📧 Onay için e-postanızı kontrol edin.')
    else:
        print(f"\n❌ {_result.get('message', 'Gönderim başarısız')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  İstek gönderildi — onay için e-postanızı kontrol edin.")
    print(f"(E-posta gelmezse tekrar deneyin veya hocaya başvurun)")
